In [7]:
import gensim
import os
import collections
import smart_open
import random

import pandas as pd
import numpy as np


In [13]:
def prepare_train_data():
    """
    SELECT
        name,
        departments_name,
        categories_name,
        description_en,
        catalog_name, -- Might not need
        price,
        bulkiness,
        is_fragile,
        is_perishable,
        weight,
        country_of_origin,
        tradename_en,
        url,
        source_product_id,
        source_brand_id,
        source_catalog_id,
        source_country_id
    FROM honeypot_v3.products
    WHERE source_country_id = 1
            AND is_published = TRUE
            AND status = 'available'
    """
    data = pd.read_csv('~/Desktop/grocery_products.csv')
    return data

In [14]:
data = prepare_train_data()

In [16]:
print(data.shape)

(893626, 17)


In [24]:
selected_data = data[['source_product_id', 'name', 'departments_name', 'categories_name', 'description_en']]
selected_data = selected_data.head(100)

selected_data.head()

,source_product_id,name,departments_name,categories_name,description_en
0,2506104,A08. Golden Nasi Super Combo,Department - (Mchefz Cuisine Menu (mchefz-cuis...,Rice,Served with fried chicken thigh and braised be...
1,2682483,Honey Stars Breakfast Cereal,Food Cupboard,Cereals & Cereal Bars,This product is getting close to its expiry da...
2,2683014,Ice Lemon Tea,Drinks,Tea,Refreshing combination of fruity flavours and ...
3,2768893,Anticavity Maximum Cavity Protection Toothpaste,Health & Beauty,Oral Care,This product is getting close to its expiry da...
4,2771143,Daily Moisture Therapy Dermo-Cleanser Twin Pack,Health & Beauty,Body Care,Daily Moisture Therapy Dermo-Cleanser is a hyp...


Creating document string

In [28]:
selected_data['document'] = selected_data['name'] + ' ' + selected_data['departments_name'] + ' ' + selected_data['categories_name'] + ' ' + selected_data['description_en']

In [36]:
selected_data.head()
documents_list = selected_data['document'].tolist()

In [33]:
selected_data[['document']].iloc[2]

document    Ice Lemon Tea Drinks Tea Refreshing combinatio...
Name: 2, dtype: object

In [37]:
documents_list

['A08. Golden Nasi Super Combo Department - (Mchefz Cuisine Menu (mchefz-cuisine)) Rice Served with fried chicken thigh and braised beef and cuttlefish, add ons available',
 "Honey Stars Breakfast Cereal Food Cupboard Cereals & Cereal Bars This product is getting close to its expiry date—don't let this get away from you! It's still safe for consumption, now enjoy it at a lower price.<br/><br/>Expiry Date: 2018-07-30<br/><br/>",
 'Ice Lemon Tea Drinks Tea Refreshing combination of fruity flavours and real tea leaves. Sweet cooling tea taste with a stimulating lemon flavour finish',
 "Anticavity Maximum Cavity Protection Toothpaste Health & Beauty Oral Care This product is getting close to its expiry date—don't let this get away from you! It's still safe for use, now enjoy it at a lower price.<br/> <br/>Expiry Date : 2018-10-13",
 'Daily Moisture Therapy Dermo-Cleanser Twin Pack Health & Beauty Body Care Daily Moisture Therapy Dermo-Cleanser is a hypoallergenic, soap-free cleanser ideal 